# 05 - Taming Text

## Deadline
Thursday December 15, 2016 at 11:59PM

## Important Notes
* Make sure you push on GitHub your Notebook with all the cells already evaluated
* Don't forget to add a textual description of your thought process, the assumptions you made, and the solution
you plan to implement!
* Please write all your comments in English, and use meaningful variable names in your code

## Background
In this homework you will explore a relatively large corpus of emails released in public during the
[Hillary Clinton email controversy](https://en.wikipedia.org/wiki/Hillary_Clinton_email_controversy).
You can find the corpus in the `hillary-clinton-emails` directory of this repository, while more detailed information 
about the [schema is available here](https://www.kaggle.com/kaggle/hillary-clinton-emails).

## Assignment
1. Generate a word cloud based on the raw corpus -- I recommend you to use the [Python word_cloud library](https://github.com/amueller/word_cloud).
With the help of `nltk` (already available in your Anaconda environment), implement a standard text pre-processing 
pipeline (e.g., tokenization, stopword removal, stemming, etc.) and generate a new word cloud. Discuss briefly the pros and
cons (if any) of the two word clouds you generated.

2. Find all the mentions of world countries in the whole corpus, using the `pycountry` utility (*HINT*: remember that
there will be different surface forms for the same country in the text, e.g., Switzerland, switzerland, CH, etc.)
Perform sentiment analysis on every email message using the demo methods in the `nltk.sentiment.util` module. Aggregate 
the polarity information of all the emails by country, and plot a histogram (ordered and colored by polarity level)
that summarizes the perception of the different countries. Repeat the aggregation + plotting steps using different demo
methods from the sentiment analysis module -- can you find substantial differences?

3. Using the `models.ldamodel` module from the [gensim library](https://radimrehurek.com/gensim/index.html), run topic
modeling over the corpus. Explore different numbers of topics (varying from 5 to 50), and settle for the parameter which
returns topics that you consider to be meaningful at first sight.

4. *BONUS*: build the communication graph (unweighted and undirected) among the different email senders and recipients
using the `NetworkX` library. Find communities in this graph with `community.best_partition(G)` method from the 
[community detection module](http://perso.crans.org/aynaud/communities/index.html). Print the most frequent 20 words used
by the email authors of each community. Do these word lists look similar to what you've produced at step 3 with LDA?
Can you identify clear discussion topics for each community? Discuss briefly the obtained results.


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import nltk
import string
import collections
nltk.download()
from os import path
from wordcloud import WordCloud
from PIL import Image
from nltk.corpus import stopwords
from nltk import word_tokenize

# 1.Generate a word cloud based on emails' content

Load email with subjects, already extracted text and raw text.

In [ ]:
raws = pd.read_csv('hillary-clinton-emails/Emails.csv',usecols=['ExtractedSubject','ExtractedBodyText','RawText'])
raws.head()

We see some NaN extractedSubject/extractedBodytext. After visualizing RawText, a majority of the cases can be explained (e.g no subject, email forwarding,..)
We decide to trust the latter and drop RawText

In [ ]:
raws.drop(['RawText'], axis= 1,inplace=True)

In [ ]:
def concat_subj_txt(row):
    subj = '' if pd.isnull(row.ExtractedSubject) else row.ExtractedSubject
    text = '' if pd.isnull(row.ExtractedBodyText) else row.ExtractedBodyText
    return subj+text

body = pd.DataFrame()
body['text'] = raws.apply(concat_subj_txt, axis=1)
body.head()

Ok. dumb word cloud with concat of all cells.

In [ ]:
text = ' '.join(body['text'])

In [ ]:
text

###  Naive Wordcloud on raw text

In [ ]:
def classic_cloud(text):
    wordcloud = WordCloud().generate(text)
    plt.figure(figsize=(10,10))
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.show()

In [ ]:
classic_cloud(text)

Not bad. Let's try something more stylish.

In [ ]:
def img_cloud(text, img):
    img = Image.open(img)
    img = img.resize((980,1080), Image.ANTIALIAS)
    hcmask = np.array(img)
    wordcloud = WordCloud(background_color="white", max_words=2000, mask=hcmask).generate(text)
    plt.figure(figsize=(10,10))
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.show()

In [ ]:
img_cloud(text, 'hc.png')

### WordCloud on clean/preprocessed data
Observe frequent words and remove the ones that are frequent and unwanted

In [ ]:
body_tokenized = [i for i in word_tokenize(text.lower())]

In [ ]:
body_tokenized

In [ ]:
stop = set(stopwords.words('english'))
stop.update(string.punctuation) #Remove ponctuation

In [ ]:
body_tokenized = [i for i in word_tokenize(text.lower()) if i not in stop]

After removing the 'english' stopwords and the punctuaction, we verify what we still need to remove

In [ ]:
counter=collections.Counter(body_tokenized)
print(counter.most_common())

We remove the unwanted words

In [ ]:
stop.update(["'s", "''", "``", 'pm', 'fw', "n't", '--']) #Remove other unwanted characters and words

In [ ]:
body_tokenized = [i for i in word_tokenize(text.lower()) if i not in stop]

In [ ]:
classic_cloud(' '.join(body_tokenized))